## Import(s)

In [3]:
from __future__ import division
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
import scipy.optimize as spopt
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Define Pose Embedder 

In [6]:
class FullBodyPoseEmbedder(object):
  """Converts 3D pose landmarks into 3D embedding."""

  def __init__(self, torso_size_multiplier=2.5):
    # Multiplier to apply to the torso to get minimal body size.
    self._torso_size_multiplier = torso_size_multiplier

    # Names of the landmarks as they appear in the prediction.
    self._landmark_names = [
        'nose',
        'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer',
        'left_ear', 'right_ear',
        'mouth_left', 'mouth_right',
        'left_shoulder', 'right_shoulder',
        'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist',
        'left_pinky_1', 'right_pinky_1',
        'left_index_1', 'right_index_1',
        'left_thumb_2', 'right_thumb_2',
        'left_hip', 'right_hip',
        'left_knee', 'right_knee',
        'left_ankle', 'right_ankle',
        'left_heel', 'right_heel',
        'left_foot_index', 'right_foot_index',
    ]

    def __call__(self, landmarks):
        """Normalizes pose landmarks and converts to embedding

        Args:
          landmarks - NumPy array with 3D landmarks of shape (N, 3).

        Result:
          Numpy array with pose embedding of shape (M, 3) where `M` is the number of
          pairwise distances defined in `_get_pose_distance_embedding`.
        """
        assert landmarks.shape[0] == len(self._landmark_names), 'Unexpected number of landmarks: {}'.format(landmarks.shape[0])

        # Get pose landmarks.
        landmarks = np.copy(landmarks)

        # Normalize landmarks.
        landmarks = self._normalize_pose_landmarks(landmarks)

        # Get embedding.
        embedding = self._get_pose_distance_embedding(landmarks)

        return embedding

    def _normalize_pose_landmarks(self, landmarks):
        """Normalizes landmarks translation and scale."""
        landmarks = np.copy(landmarks)

        # Normalize translation.
        pose_center = self._get_pose_center(landmarks)
        landmarks -= pose_center

        # Normalize scale.
        pose_size = self._get_pose_size(landmarks, self._torso_size_multiplier)
        landmarks /= pose_size
        # Multiplication by 100 is not required, but makes it eaasier to debug.
        landmarks *= 100

        return landmarks

    def _get_pose_center(self, landmarks):
        """Calculates pose center as point between hips."""
        left_hip = landmarks[self._landmark_names.index('left_hip')]
        right_hip = landmarks[self._landmark_names.index('right_hip')]
        center = (left_hip + right_hip) * 0.5
        return center

    def _get_pose_size(self, landmarks, torso_size_multiplier):
        """Calculates pose size.

        It is the maximum of two values:
          * Torso size multiplied by `torso_size_multiplier`
          * Maximum distance from pose center to any pose landmark
        """
        # This approach uses only 2D landmarks to compute pose size.
        landmarks = landmarks[:, :2]

        # Hips center.
        left_hip = landmarks[self._landmark_names.index('left_hip')]
        right_hip = landmarks[self._landmark_names.index('right_hip')]
        hips = (left_hip + right_hip) * 0.5

        # Shoulders center.
        left_shoulder = landmarks[self._landmark_names.index('left_shoulder')]
        right_shoulder = landmarks[self._landmark_names.index('right_shoulder')]
        shoulders = (left_shoulder + right_shoulder) * 0.5

        # Torso size as the minimum body size.
        torso_size = np.linalg.norm(shoulders - hips)

        # Max dist to pose center.
        pose_center = self._get_pose_center(landmarks)
        max_dist = np.max(np.linalg.norm(landmarks - pose_center, axis=1))

        return max(torso_size * torso_size_multiplier, max_dist)

    def _get_pose_distance_embedding(self, landmarks):
        """Converts pose landmarks into 3D embedding.

        We use several pairwise 3D distances to form pose embedding. All distances
        include X and Y components with sign. We differnt types of pairs to cover
        different pose classes. Feel free to remove some or add new.

        Args:
          landmarks - NumPy array with 3D landmarks of shape (N, 3).

        Result:
          Numpy array with pose embedding of shape (M, 3) where `M` is the number of
          pairwise distances.
        """
        embedding = np.array([
            # One joint.

            self._get_distance(
                self._get_average_by_names(landmarks, 'left_hip', 'right_hip'),
                self._get_average_by_names(landmarks, 'left_shoulder', 'right_shoulder')),

            self._get_distance_by_names(landmarks, 'left_shoulder', 'left_elbow'),
            self._get_distance_by_names(landmarks, 'right_shoulder', 'right_elbow'),

            self._get_distance_by_names(landmarks, 'left_elbow', 'left_wrist'),
            self._get_distance_by_names(landmarks, 'right_elbow', 'right_wrist'),

            self._get_distance_by_names(landmarks, 'left_hip', 'left_knee'),
            self._get_distance_by_names(landmarks, 'right_hip', 'right_knee'),

            self._get_distance_by_names(landmarks, 'left_knee', 'left_ankle'),
            self._get_distance_by_names(landmarks, 'right_knee', 'right_ankle'),

            # Two joints.

            self._get_distance_by_names(landmarks, 'left_shoulder', 'left_wrist'),
            self._get_distance_by_names(landmarks, 'right_shoulder', 'right_wrist'),

            self._get_distance_by_names(landmarks, 'left_hip', 'left_ankle'),
            self._get_distance_by_names(landmarks, 'right_hip', 'right_ankle'),

            # Four joints.

            self._get_distance_by_names(landmarks, 'left_hip', 'left_wrist'),
            self._get_distance_by_names(landmarks, 'right_hip', 'right_wrist'),

            # Five joints.

            self._get_distance_by_names(landmarks, 'left_shoulder', 'left_ankle'),
            self._get_distance_by_names(landmarks, 'right_shoulder', 'right_ankle'),

            self._get_distance_by_names(landmarks, 'left_hip', 'left_wrist'),
            self._get_distance_by_names(landmarks, 'right_hip', 'right_wrist'),

            # Cross body.

            self._get_distance_by_names(landmarks, 'left_elbow', 'right_elbow'),
            self._get_distance_by_names(landmarks, 'left_knee', 'right_knee'),

            self._get_distance_by_names(landmarks, 'left_wrist', 'right_wrist'),
            self._get_distance_by_names(landmarks, 'left_ankle', 'right_ankle'),

            # Body bent direction.

            # self._get_distance(
            #     self._get_average_by_names(landmarks, 'left_wrist', 'left_ankle'),
            #     landmarks[self._landmark_names.index('left_hip')]),
            # self._get_distance(
            #     self._get_average_by_names(landmarks, 'right_wrist', 'right_ankle'),
            #     landmarks[self._landmark_names.index('right_hip')]),
        ])

        return embedding

    def _get_average_by_names(self, landmarks, name_from, name_to):
        lmk_from = landmarks[self._landmark_names.index(name_from)]
        lmk_to = landmarks[self._landmark_names.index(name_to)]
        return (lmk_from + lmk_to) * 0.5

    def _get_distance_by_names(self, landmarks, name_from, name_to):
        lmk_from = landmarks[self._landmark_names.index(name_from)]
        lmk_to = landmarks[self._landmark_names.index(name_to)]
        return self._get_distance(lmk_from, lmk_to)

    def _get_distance(self, lmk_from, lmk_to):
        return lmk_to - lmk_from

## Load Data

In [ ]:
path = "set/path/to/location/of/csv/output"

In [ ]:
# Update classes and class map to reflect pose set
classes = ['circle','handsShoulders','sideT','hips','Vdown','track'] 
class_map = {1:'circle',2:'handsShoulders',3:'sideT',4:'hips',5:'Vdown',6:'track'}

class_num = 1
merge = []
for cl in classes:
    df = pd.read_csv(f'{path}/{cl}.csv',header=None) # Put absolute path
    df['Class'] = class_num
    class_num+=1
    merge.append(df)
data_final = pd.concat(merge,ignore_index=True)

# Filter Data
filt = [x for x in data_final.columns if x in range(1,102,1)] + ['Class']
data_and_labels = data_final[filt]
for i in range(10):
    data_and_labels = data_and_labels.sample(frac=1) # shuffle data

# Reshape data and create embedding
data_filt = [x for x in data_and_labels.columns if x in range(1,100,1)]
data = data_and_labels[data_filt]
data = data.to_numpy()
embed = FullBodyPoseEmbedder()
arrays = []
for i in range(len(data)):
    temp = data[i,:] # isolate single example
    temp = temp.reshape((33,3)) # reshape to landmark output dimensions
    temp = embed(temp)
    temp = temp.flatten()
    arrays.append(temp)
data = np.stack(arrays, axis=0)

# break up data
train_data = data[0:int(len(data)*.7)]
test_data = data[int(len(data)*.7):]
label_filt = ['Class']
labels = data_and_labels[label_filt]
labels = labels.to_numpy()
labels = labels.reshape(len(labels),)
train_labels = labels[0:int(len(data)*.7)]
test_labels = labels[int(len(data)*.7):]
print(np.unique(train_labels))
print(np.unique(test_labels))

## PCA

In [ ]:
# PCA
# Separate the digits into labels
inds=np.array([train_labels==i for i in range(len(classes))])

from sklearn.decomposition import PCA
pca = PCA(n_components=20, whiten=True)
pca.fit(train_data)
sk_pose_reduced=pca.transform(train_data)
plt.figure(figsize=(10,10))
for i in range(len(classes)):
    plt.plot(sk_pose_reduced[inds[i],0],
             sk_pose_reduced[inds[i],1],
             'o', label=str(class_map[i+1]), alpha=0.5)
plt.gca().legend()
plt.show()

## Regression 

In [ ]:
logisticRegr = LogisticRegression(max_iter=1000)
logisticRegr.fit(train_data, train_labels) # if the algorithm did not converge, try increase iteration number by specifying "max_iter"

s = time.time()
print(logisticRegr.predict(test_data[0].reshape(1,-1))[0])
print(test_labels[0])
e = time.time()
print(e-s)

# Make predictions on entire test data
predictions = logisticRegr.predict(test_data)
score = logisticRegr.score(test_data, test_labels)
print(score)

## Save Classifier as .pkl file

In [ ]:
import pickle
pkl_filename = f"{path}/LogReg_pose_classifier.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(logisticRegr, file)